# Bygge AI-agenter med vedvarende minne ved hjelp av Cognee

Denne notatboken viser hvordan man kan bygge intelligente AI-agenter med avanserte minnefunksjoner ved hjelp av [**cognee**](https://www.cognee.ai/) - en åpen kildekode AI-minne som kombinerer kunnskapsgrafer, semantisk søk og sesjonsstyring for å skape kontekstbevisste AI-systemer.

## 🎯 Læringsmål

Ved slutten av denne veiledningen vil du forstå hvordan du:
- **Bygger kunnskapsgrafer støttet av embeddings**: Gjør ustrukturert tekst om til strukturert, søkbar kunnskap
- **Implementerer sesjonsminne**: Opprett flervendte samtaler med automatisk kontekstbevaring
- **Lagrer samtaler**: Valgfritt lagre viktige interaksjoner i langtidsminne for fremtidig bruk
- **Utfører spørringer med naturlig språk**: Få tilgang til og utnytt historisk kontekst i nye samtaler
- **Visualiserer minne**: Utforsk relasjonene i agentens kunnskapsgraf


## 🏗️ Hva du skal bygge

I denne veiledningen skal vi lage en **Kodeassistent** med vedvarende minne som:

### 1. **Konstruksjon av kunnskapsbase**
   - Tar inn informasjon om utviklerprofil og ekspertise
   - Behandler prinsipper og beste praksis for Python-programmering
   - Lagrer historiske samtaler mellom utviklere og AI-assistenter

### 2. **Samtaler med sesjonskontekst**
   - Opprettholder kontekst på tvers av flere spørsmål i samme sesjon
   - Automatisk lagrer hvert spørsmål/svar-par for effektiv gjenfinning
   - Gir sammenhengende, kontekstuelle svar basert på samtalehistorikk

### 3. **Langtidsminne**
   - Lagrer viktige samtaler i et langtidsminne
   - Henter relevante minner fra kunnskapsbasen og tidligere sesjoner for å informere nye interaksjoner
   - Bygger en voksende kunnskapsbase som forbedres over tid

### 4. **Intelligent minnehenting**
   - Bruker grafbevisst semantisk søk for å finne relevant informasjon på tvers av all lagret kunnskap
   - Filtrerer søk etter datasubgrupper (utviklerinfo vs. prinsipper)
   - Kombinerer flere datakilder for å gi omfattende svar


## 📋 Forutsetninger og oppsett

### Systemkrav

Før du starter, sørg for at du har:

1. **Python-miljø**
   - Python 3.9 eller nyere
   - Virtuelt miljø (anbefales)
   
2. **Redis Cache** (Påkrevd for sesjonshåndtering)
   - Lokal Redis: `docker run -d -p 6379:6379 redis`
   - Eller bruk en administrert Redis-tjeneste
   
3. **LLM API-tilgang**
   - OpenAI API-nøkkel eller andre leverandører (se [dokumentasjon](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Databasekonfigurasjon**
   - Ingen konfigurasjon nødvendig som standard. Cognee bruker filbaserte databaser (LanceDB og Kuzu)
   - Valgfritt kan du sette opp Azure AI Search som en vektorlagring (se [dokumentasjon](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Miljøkonfigurasjon

Opprett en `.env`-fil i prosjektmappen din med følgende variabler:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Forstå Cognees minnearkitektur

### Hvordan Cognee fungerer

Cognee tilbyr et avansert minnssystem som går utover enkel nøkkel-verdi-lagring:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Hovedkomponenter:

1. **Kunnskapsgraf**: Lagrer enheter, relasjoner og semantiske forbindelser
2. **Vektorinbedding**: Muliggjør semantisk søk på tvers av all lagret informasjon
3. **Sesjonsbuffer**: Opprettholder samtalekontekst innenfor og på tvers av sesjoner
4. **NodeSets**: Organiserer data i logiske kategorier for målrettet gjenfinning

### Minne-typer i denne veiledningen:

- **Vedvarende minne**: Langtidslagring i kunnskapsgrafen
- **Sesjonsminne**: Midlertidig samtalekontekst i Redis-buffer
- **Semantisk minne**: Vektorbasert likhetssøk på tvers av all data


## 📦 Installer nødvendige pakker

Installer Cognee med Redis-støtte for øktadministrasjon:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Initialiser miljøet og last inn biblioteker

Sørg for at:
1. Redis kjører (f.eks. via Docker: `docker run -d -p 6379:6379 redis`)
2. Miljøvariabler er satt før du importerer cache-moduler
3. Om nødvendig, start kjernen på nytt og kjør cellene i rekkefølge

Den følgende cellen vil:
1. Laste inn miljøvariabler fra `.env`
2. Konfigurere Cognee med dine LLM-innstillinger
3. Aktivere caching for øktstyring
4. Validere at alle komponenter er riktig tilkoblet


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigurer lagringskataloger

Cognee bruker to separate kataloger for sine operasjoner:
- **Data Root**: Lagrer importerte dokumenter og bearbeidede data
- **System Root**: Inneholder kunnskapsgrafdatabasen og systemmetadata

Vi skal opprette isolerte kataloger for denne veiledningen som følger:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Tilbakestill minnetilstand

Før vi begynner å bygge vårt minnesystem, la oss sørge for at vi starter med blanke ark.

> 💡 **Tips**: Du kan hoppe over dette trinnet hvis du ønsker å bevare eksisterende minner fra tidligere kjøringer når du bruker denne notatboken senere.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Del 1: Bygge kunnskapsbasen

### Datakilder for vår utviklerassistent

Vi vil bruke tre typer data for å lage en omfattende kunnskapsbase:

1. **Utviklerprofil**: Personlig ekspertise og teknisk bakgrunn
2. **Beste praksis for Python**: The Zen of Python med praktiske retningslinjer
3. **Historiske samtaler**: Tidligere spørsmål og svar mellom utviklere og AI-assistenter

Denne varierte dataen gjør det mulig for agenten vår å:
- Forstå brukerens tekniske kontekst
- Anvende beste praksis i anbefalinger
- Lære av tidligere vellykkede interaksjoner


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Prosesser data til kunnskapsgraf

Nå skal vi transformere råteksten vår til en strukturert hukommelse. Denne prosessen:

1. **Legger data til NodeSets**: Organiserer informasjon i logiske kategorier
   - `developer_data`: Utviklerprofil og samtaler
   - `principles_data`: Beste praksis og retningslinjer for Python

2. **Kjører Cognify Pipeline**: Ekstraherer enheter, relasjoner og lager embeddings
   - Identifiserer nøkkelkonsepter
   - Skaper semantiske forbindelser mellom relatert informasjon
   - Genererer vektorembeddings

Dette kan ta noen øyeblikk mens LLM prosesserer teksten og bygger grafstrukturen:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualiser kunnskapsgrafen

La oss utforske strukturen til kunnskapsgrafen vår. Visualiseringen viser:
- **Noder**: Enheter hentet fra teksten (konsepter, teknologier, personer)
- **Kanter**: Forhold og forbindelser mellom enheter
- **Klynger**: Relaterte konsepter gruppert etter semantisk likhet

Åpne den genererte HTML-filen i nettleseren din for å utforske grafen interaktivt:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Berik hukommelsen med Memify

Funksjonen `memify()` analyserer kunnskapsgrafen og genererer intelligente regler om dataene. Denne prosessen:
- Identifiserer mønstre og beste praksis
- Lager handlingsrettede retningslinjer basert på innholdet
- Etablerer sammenhenger mellom ulike kunnskapsområder

Disse reglene hjelper agenten med å ta mer informerte beslutninger når den svarer på spørsmål. Å fange en andre visualisering hjelper deg med å sammenligne hvordan grafen blir tettere når den er beriket.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Del 2: Intelligent minnehenting

### Demonstrasjon 1: Tverrdokument kunnskapsintegrasjon

Nå som vår kunnskapsgraf er bygget, la oss teste hvordan Cognee kombinerer informasjon fra flere kilder for å svare på komplekse spørsmål.

Den første forespørselen demonstrerer:
- **Semantisk forståelse**: Finne relevante konsepter selv når de ikke er eksplisitt nevnt
- **Tverrhenvisning**: Kombinere utviklerprofil med Python-prinsipper
- **Kontekstuell resonnement**: Anvende beste praksis på spesifikke implementeringer

### Demonstrasjon 2: Filtrert søk med NodeSets

Den andre forespørselen viser hvordan man kan målrette spesifikke delsett av kunnskapsgrafen:
- Bruker `node_name`-parameteren for å søke kun innenfor `principles_data`
- Gir fokuserte svar fra et spesifikt kunnskapsområde
- Nyttig når du trenger informasjon som er spesifikk for et domene


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Del 3: Oppsett av øktadministrasjon

### Aktivere samtaleminne

Øktadministrasjon er avgjørende for å opprettholde kontekst gjennom flere interaksjoner. Her skal vi:

1. **Initialisere brukerkontekst**: Opprette eller hente en brukerprofil for sporing av økter
2. **Konfigurere cache-motor**: Koble til Redis for lagring av samtalehistorikk
3. **Aktivere øktvariabler**: Sette opp kontekstvariabler som vedvarer gjennom forespørsler

> ⚠️ **Viktig**: Dette krever at Redis kjører og `CACHING=true` i ditt miljø


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Hjelpefunksjon: Vis Økt Historikk

Denne hjelpefunksjonen lar oss inspisere samtalehistorikken som er lagret i Redis. Den er nyttig for:
- Feilsøking av økthåndtering
- Verifisere at samtaler blir bufret
- Forstå hvilken kontekst som er tilgjengelig for agenten


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesjon 1: Async Support Lab — Første Spørsmål

Start `async-support-lab`-sesjonen ved å spørre om telemetrivennlige asyncio-mønstre for en massiv webskraper. Grafen kjenner allerede til asyncio, aiohttp og overvåkingspraksis, så svaret bør gjenspeile tidligere samtaler samtidig som det tilpasses det nye spørsmålet.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspiser minnet for sesjon 1 etter den første utvekslingen

Å kjøre `show_history(session_1)` umiddelbart etter det innledende spørsmålet bekrefter at Cognee skrev både prompten og fullføringen inn i Redis. Du bør se én oppføring med veiledning for samtidighet.


In [ ]:
await show_history(session_1)

## Økt 1: Oppfølging på datamodeller

Deretter spør vi: "Når bør jeg velge dataclasses fremfor Pydantic?" ved å bruke samme økt-ID. Cognee bør sette sammen Python-prinsippene og tidligere FastAPI-samtaler for å gi nyansert råd—og vise at konteksten videreføres innenfor en navngitt økt.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Bekreft at historikken for økt 1 inneholder begge turene

En ny `show_history(session_1)`-kall bør vise to spørsmål-og-svar-oppføringer. Dette samsvarer med Mem0-laboratoriets "minnegjennomspilling"-steg og beviser at flere turer utvider samme transkripsjon.


In [ ]:
await show_history(session_1)

## Økt 2: Designgjennomgangstråd — Ny økt

For å vise isolasjon mellom tråder starter vi `design-review-session` og ber om veiledning for logging av hendelsesgjennomganger. Selv om den underliggende kunnskapsbasen er den samme, holder den nye økt-ID-en transkripsjonene adskilt.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Gjennomgang av økt 2 Historikk

`show_history(session_2)` skal kun vise design-gjennomgangens prompt/svar-par. Sammenlign det med økt 1 for å fremheve hvordan Cognee holder uavhengige transkripsjoner samtidig som den gjenbruker den delte kunnskapsgrafen.


In [ ]:
await show_history(session_2)

## Sammendrag

Gratulerer! Du har nettopp gitt din kodeassistent et ekte langtidsminnelag drevet av Cognee.

I denne veiledningen tok du rått utviklerinnhold (kode, dokumentasjon, samtaler) og gjorde det om til en graf + vektorminne som agenten din kan søke i, resonnere over og kontinuerlig forbedre.

Hva du har lært

1. **Fra råtekst til AI-minne**: Hvordan Cognee tar inn ustrukturert data og gjør det om til intelligent, søkbart minne ved hjelp av en kombinert vektor- og kunnskapsgrafarkitektur.

2. **Grafberikelse med memify**: Hvordan gå utover grunnleggende grafopprettelse og bruke memify til å legge til avledede fakta og rikere relasjoner på toppen av din eksisterende graf.

3. **Flere søkestrategier**: Hvordan søke i minnet med ulike søketyper (grafbevisst Q&A, RAG-stil fullføring, innsikter, rå tekstbiter, kodesøk, osv.) avhengig av hva agenten din trenger.

4. **Visuell utforskning**: Hvordan inspisere og feilsøke det Cognee har bygget ved hjelp av grafvisualiseringer og Cognee UI, slik at du faktisk kan se hvordan kunnskapen er strukturert.

5. **Øktbevisst minne**: Hvordan kombinere kontekst fra hver økt med vedvarende semantisk minne, slik at agenter kan huske på tvers av økter uten å lekke informasjon mellom brukere.


## Viktige punkter
1. Minne som en kunnskapsgraf støttet av embeddings

    - **Strukturert forståelse**: Cognee kombinerer en vektorlager og en grafdatabase slik at dataene dine både er søkbare etter mening og koblet gjennom relasjoner. Cognee bruker filbaserte databaser som standard (LanceDB for vektor-, Kuzu for grafdatabase).

    - **Relasjonsbevisst gjenfinning**: Svar kan baseres ikke bare på "lignende tekst", men også på hvordan enheter er relatert.

    - **Levende minne**: Minnelaget utvikler seg, vokser og forblir søkbart som en sammenkoblet graf.

2. Søke- og resonneringsmoduser
    - **Hybrid gjenfinning**: Søket kombinerer vektorsimilaritet, grafstruktur og LLM-resonnering, fra rå datablokk-oppslag til grafbevisst spørsmål-svar.

    - **Tilpass modus til oppgaven**: Bruk fullføringsmoduser når du ønsker naturlige språkbaserte svar, og blokk-/sammendrags-/grafmoduser når agenten din trenger rå kontekst eller skal drive sin egen resonnering.

3. Personlige, sesjonsbevisste agenter
    - **Sesjonskontekst + langtidsminne**: Cognee holder kortsiktig "tråd"-kontekst adskilt fra langvarig, bruker- eller organisasjonsnivåminne.

## Virkelige bruksområder

1. **Vertikale AI-agenter**

    Bruk mønsteret fra denne notatboken for å drive domenespesifikke copiloter som bruker Cognee som sin kjerne for gjenfinning og resonnering:

- **Utvikler-copiloter**: Kodegjennomgang, hendelsesanalyse og arkitekturassistenter som navigerer gjennom kode, API-er, design-dokumenter og billetter som en enkelt minnegraf.

- **Kundeorienterte copiloter**: Støtte- eller suksessagenter som henter informasjon fra produktdokumenter, vanlige spørsmål, CRM-notater og tidligere billetter med grafbevisst gjenfinning og siterte svar.

- **Interne ekspert-copiloter**: Policy-, juridiske- eller sikkerhetsassistenter som resonerer over sammenkoblede regler, retningslinjer og historiske beslutninger i stedet for isolerte PDF-er.

    Cognee er eksplisitt posisjonert som et vedvarende, nøyaktig minne for AI-agenter, og tilbyr en levende kunnskapsgraf som kan integreres bak agenten din og erstatte ad-hoc kombinasjoner av vektorlager og tilpasset grafkode.

2. **Forene datasiloer til ett minne**

    Den samme tilnærmingen hjelper deg også med å bygge et samlet minnelag på tvers av spredte kilder:

- **Fra siloer til én graf**: Importer strukturerte (f.eks. databaser) og ustrukturerte data (f.eks. dokumenter, chatter) til en enkelt graf støttet av embeddings, i stedet for separate indekser for hvert system.

- **Tverrkilde-resonnering med siteringer**: Utfør flerstegsresonnering over alt—"koble sammen" logger, målinger og dokumenter via grafen—og returner fortsatt begrunnede svar med kildehenvisning.

- **Kunnskapshuber**: For domener som bank eller utdanning brukes Cognee allerede til å forene PDF-er, interne systemer og appdata i én kunnskapsgraf med vektorer, slik at agenter kan svare på spørsmål med presis, sitert kontekst.

## Neste steg

Du har implementert den grunnleggende minnesløyfen. Her er naturlige utvidelser du kan prøve på egen hånd (se [Cognee-dokumentasjon](https://docs.cognee.ai/) for detaljer):

1. **Eksperimenter med tidsbevissthet**: Aktiver tidsmessig cognify for å trekke ut hendelser og tidsstempler fra tekst.

2. **Introduser ontologidrevet resonnering**: Definer en OWL-ontologi for ditt domene. Bruk Cognee sin ontologistøtte slik at uttrukne enheter og relasjoner er forankret i det skjemaet, noe som forbedrer grafkvaliteten og domenespesifikke svar.

3. **Legg til en tilbakemeldingssløyfe**: La Cognee justere grafkantvekter basert på reelle brukertilbakemeldinger, slik at gjenfinning forbedres over tid i stedet for å forbli statisk.

4. **Tilpass for personalisering og sesjonsatferd**: Bruk bruker-ID-er, leietakere og datasett for å gi hver person eller team sitt eget syn på den delte minnemotoren.

5. **Skaler opp til mer komplekse agenter**: Koble Cognee til agentrammeverk for å bygge multi-agent-systemer som alle deler det samme minnelaget. *Microsoft Agent Framework x Cognee-plugin kommer snart.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
